In [ ]:
import dolfinx as dfx

import h5py

from mpi4py import MPI

import numpy as np

import pyvista as pv

from pyMoBiMP.plotting_utils import PyvistaAnimation

In [ ]:
file = "../simulation_output/CH_4_true_3d"

with h5py.File(file + ".h5", 'r') as f:
    print(f.keys())

    # grid coordinates
    x_data = f["Mesh/mesh/geometry"][()]

    # time steps (convert from string to float)
    t = [float(t.replace("_", ".")) for t in f["Function/y"].keys()]

    # list of data stored as numpy arrays
    u_data = [(f["Function/y"][u_key][()].squeeze(),
               f["Function/mu"][u_key][()].squeeze())
              for u_key in f["Function/y"]]

sorted_indx = np.argsort(t)

t = np.array(t)[sorted_indx]
u_data = np.array(u_data)[sorted_indx]

# rt_data  = np.loadtxt(results_folder / "CH_4_min_1D_rt.txt")

In [ ]:
with dfx.io.XDMFFile(MPI.COMM_WORLD, file + ".xdmf", 'r') as xdmf:
    mesh = xdmf.read_mesh()

In [ ]:
plotter = pv.Plotter()

V = dfx.fem.FunctionSpace(mesh, ("Lagrange", 1))

topo, ct, x = dfx.plot.vtk_mesh(V)

grid = pv.UnstructuredGrid(topo, ct, x)

y = u_data[2, 0, :]
c = np.exp(y) / (1 + np.exp(y))

grid["u"] = c

x0 = (0, 0, 0)
grid = grid.clip_box([x0[0], x0[0]+1, x0[1], x0[1]+1, 0, 0+1])

plotter.add_mesh(grid)

plotter.show()